In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import random
import os

print(torch.get_num_threads())
print(torch.get_num_interop_threads())

32
32


In [2]:
#x_train_orig = np.arange(-2.4, 2.8, 0.4)
#x_train_orig = np.arange(-2.4, 2.6, 0.2)
#x_train_orig = np.arange(-2.4, 2.5, 0.1)
x_train_orig = np.arange(-2.4, 2.45, 0.05)
#x_train_orig = np.arange(-2.4, 2.425, 0.025)

In [3]:
class FullModel(torch.nn.Module):
    def __init__(self):
        super(FullModel, self).__init__()
        self.fc1 = torch.nn.Linear(1, 8)
        self.fc2 = torch.nn.Linear(8, 8)
        self.fc3 = torch.nn.Linear(8, 1)
        self.relu = torch.nn.ReLU()
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

In [4]:
def func(x):
    x = x.astype(float)
    return np.power(x, 1) * np.power(np.sin(1.0 / 3.0 * x), 2)

def avg_l2_diff(y1, y2):
    return np.average(np.power(y1-y2, 2))

In [5]:
def train(model, optimizer, criterion, epoch):
    model.train()
    
    running_loss  = torch.tensor(0.0)

    for batch_idx, current_batch in enumerate(train_loader):     
        inp, current_batch_y = current_batch[0],        current_batch[1]

        optimizer.zero_grad()
        output = model(inp)
        gndtruth = current_batch_y

        loss = criterion(output, gndtruth)
        loss.backward()
        optimizer.step()
        running_loss  += loss.item()

    running_loss  = running_loss  / len(train_loader)
    
    if epoch % 40 == 0:
        print("Epoch: {}, Average loss: {:15.8f}".format(epoch, running_loss))

In [6]:
l2_diff_tot_before = []
l2_diff_tot_after = []
for seed in np.arange(20):
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.use_deterministic_algorithms(True)
    
    model = FullModel()

    x_train = x_train_orig
    print("Orig train size = ", x_train.shape[0])
    nsample = x_train.shape[0] - 1

    y_train = func(x_train)
    x_train_torch = torch.from_numpy(x_train).float()
    y_train_torch = torch.from_numpy(y_train).float()
    train_dataset = torch.utils.data.TensorDataset(x_train_torch, y_train_torch)
    train_loader = torch.utils.data.DataLoader(
        train_dataset, shuffle=True)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.02)
    criterion = torch.nn.MSELoss()
    
    for epoch in range(1, 2000):
        train(model, optimizer, criterion, epoch)
        
    x_pred_np = np.arange(-2.5, 2.5001, 0.0001).reshape(-1, 1)
    x_pred = torch.from_numpy(x_pred_np).float()
    y_true = func(x_pred_np)
    y_pred = model(x_pred)
    x_np = x_pred.numpy().reshape(-1)
    y_pred_np = y_pred.detach().numpy().reshape(-1)
    y_true_np = y_true.reshape(-1)
    l2_diff = avg_l2_diff(y_pred_np, y_true_np).astype(float)
    l2_diff_tot_before.append(l2_diff)
    
    
    rng = np.random.default_rng(seed)
    res = np.empty([0])
    w = np.power(y_pred_np - y_true_np, 2)
    w = w / np.sum(w)
    sz = w.shape[0]
    freq = rng.multinomial(nsample, w)

    for i in range(sz):
        if freq[i]:
            res = np.concatenate((res, [x_np[i]]), axis=0)
    print(res)    
    
    x_train = x_train_orig
    x_train = np.concatenate((x_train, res), axis=0)
    print("After AL train size = ", x_train.shape[0])
    y_train = func(x_train)
    x_train_torch = torch.from_numpy(x_train).float()
    y_train_torch = torch.from_numpy(y_train).float()
    train_dataset = torch.utils.data.TensorDataset(x_train_torch, y_train_torch)
    train_loader = torch.utils.data.DataLoader(
        train_dataset, shuffle=True)

    for epoch in range(1, 2000):
        train(model, optimizer, criterion, epoch)

    x_pred_np = np.arange(-2.5, 2.5001, 0.0001).reshape(-1, 1)
    x_pred = torch.from_numpy(x_pred_np).float() 
    y_true = func(x_pred_np)
    y_pred = model(x_pred)
    x_np = x_pred.numpy().reshape(-1)
    y_pred_np = y_pred.detach().numpy().reshape(-1)
    y_true_np = y_true.reshape(-1)  
    l2_diff = avg_l2_diff(y_pred_np, y_true_np).astype(float)
    l2_diff_tot_after.append(l2_diff)

Orig train size =  97
Epoch: 40, Average loss:      0.00149046
Epoch: 80, Average loss:      0.00087699
Epoch: 120, Average loss:      0.00064599
Epoch: 160, Average loss:      0.00061289
Epoch: 200, Average loss:      0.00051310
Epoch: 240, Average loss:      0.00049952
Epoch: 280, Average loss:      0.00045585
Epoch: 320, Average loss:      0.00041563
Epoch: 360, Average loss:      0.00037952
Epoch: 400, Average loss:      0.00034741
Epoch: 440, Average loss:      0.00032372
Epoch: 480, Average loss:      0.00027444
Epoch: 520, Average loss:      0.00026596
Epoch: 560, Average loss:      0.00025768
Epoch: 600, Average loss:      0.00026465
Epoch: 640, Average loss:      0.00024032
Epoch: 680, Average loss:      0.00021388
Epoch: 720, Average loss:      0.00022984
Epoch: 760, Average loss:      0.00019719
Epoch: 800, Average loss:      0.00022691
Epoch: 840, Average loss:      0.00020231
Epoch: 880, Average loss:      0.00020716
Epoch: 920, Average loss:      0.00018590
Epoch: 960, Av

In [7]:
print(l2_diff_tot_before)
print(l2_diff_tot_after)

[0.0001843994118904418, 0.000140458339364572, 0.00018331123883348312, 9.050157696249321e-05, 0.00010419244845061937, 0.00016407958283556675, 0.0002280552344549508, 0.0002792043562325209, 8.86037769908846e-05, 0.00019691054705556652, 0.0002493472204808926, 0.00018443140266941572, 0.00044009315512983055, 9.247560630576441e-05, 7.516297060815092e-05, 0.00023501752435911628, 0.00010828435921555394, 0.00021625843862991802, 0.0010034739279394782, 0.00010649518814399633]
[3.82243764680905e-05, 4.105195281983206e-05, 0.00010570941958046351, 4.471470955684322e-05, 4.1809718216518924e-05, 0.00011507978864216079, 0.00018715153570943163, 0.00022533810435884495, 9.191853067472009e-05, 0.00013809626562314353, 9.208904585718241e-05, 9.776118112371016e-05, 0.00041554330337589276, 7.140966402535046e-05, 3.33169827587556e-05, 0.0001847325342000943, 5.703283120746374e-05, 0.00028633744644201336, 0.0012171180570015507, 4.734883955191576e-05]
